In [1]:
HTML(read(open("style.html"), String))

HTML{String}("<link href='https://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('https://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\", sans-serif;\r\n    font-size: 100%;\r\n}\r\n\r\n</style>")

In [ ]:
include("silent.jl")

In [2]:
using Pkg
@silent Pkg.add("NBInclude")
using NBInclude

In [3]:
@nbinclude "2.1 - Board.ipynb"
@nbinclude "3.1 - Evaluation.ipynb"

evaluateEPCaptureScoreDelta (generic function with 1 method)

***

Der folgenden Tests validieren, dass stehts die Logik `undomove(domove(board)) = board` gilt.

In [4]:
extboard = ExtendedBoard(Chess.startboard())
original = deepcopy(extboard)

extundo = domove!(extboard, Chess.movefromstring("e2e4"))
undomove!(extboard, extundo)

Chess.undomove!(original.board, Chess.domove!(original.board, Chess.movefromstring("e2e4")))

# Chess.Board objects cannot be compared directly
@assert string(extboard.board) == string(original.board)
@assert extboard.score == original.score
@assert extboard.endGame == original.endGame
# Undo-Operation decrements a counter, so we have to remove entries with a counter value of 0
filter!(entry -> entry.second != 0, extboard.repetitions)
@assert extboard.repetitions == original.repetitions
@assert extboard.repetitionRuleDraw == original.repetitionRuleDraw
@assert extboard.zobrist.hash == original.zobrist.hash

In [5]:
for x in 1:1000

    extboard = ExtendedBoard(Chess.startboard())

    while !Chess.isterminal(extboard.board) && !extboard.repetitionRuleDraw

        original = deepcopy(extboard)

        move = rand(Chess.moves(extboard.board))
        extundo = domove!(extboard, move)
        undomove!(extboard, extundo)

        Chess.undomove!(original.board, Chess.domove!(original.board, move))

        # Chess.Board objects cannot be compared directly
        @assert string(extboard.board) == string(original.board)
        @assert extboard.score == original.score
        @assert extboard.endGame == original.endGame
        # Undo-Operation decrements a counter, so we have to remove entries with a counter value of 0
        filter!(entry -> entry.second != 0, extboard.repetitions)
        @assert extboard.repetitions == original.repetitions
        @assert extboard.repetitionRuleDraw == original.repetitionRuleDraw
        @assert extboard.zobrist.hash == original.zobrist.hash

        domove!(extboard, move)
    end
    
    if x % 100 == 0
        println("$((x / 100) * 10)%")
    end
end

10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
100.0%


Der folgende Test validiert, dass die inkrementellen Implementierungen der Evaluierungsfunktion, sowie des Zobrist-Hashings stehts die gleichen Werte wie ihre iterativen Versionen ergeben. Hierbei wird ein Schachspiel durchgespielt, bei welchem nach jedem Zug die erwarteten Werte mit den inkrementell errechneten Werten vergleichen werden. Insgesammt wird der Test `10000x` wiederholt, um alle möglichen Randbedingungen und Spezialfälle abzudecken.

In [6]:
for x in 1:10000
    extboard = ExtendedBoard(Chess.startboard())

    # deepcopy to get the same random hash keys
    zobristCopy = deepcopy(extboard.zobrist)

    while !Chess.isterminal(extboard.board) && !extboard.repetitionRuleDraw

        expectedIsEndGame = isEndGame(extboard.board)
        @assert extboard.endGame == expectedIsEndGame

        expectedScore = evaluatePositionScore(extboard.board, 
            expectedIsEndGame ? PIECE_SQUARE_TABLES_ENDGAME : PIECE_SQUARE_TABLES_MIDGAME)
        @assert expectedScore == extboard.score

        rehash!(zobristCopy, extboard.board)
        @assert extboard.zobrist.hash == zobristCopy.hash

        move = rand(Chess.moves(extboard.board))
        domove!(extboard, move)
    end
    
    if x % 1000 == 0
        println("$((x / 1000) * 10)%")
    end
end

10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
100.0%


Der folgenden Tests validieren, dass das dreifache Wiederholen einer Stellung zu einem Unentschieden führt.

In [7]:
board = Chess.fromfen("rnb1kbnr/pppp1ppp/5q2/8/3QP3/2P5/PP3PPP/RNB1KBNR w KQkq -")
extboard = ExtendedBoard(board)

println("Initial State")
IJulia.display(HTML(IJulia.html(board)))

# first visit of position -> ok
@assert !extboard.repetitionRuleDraw 

# white move, black move
domove!(extboard, Chess.movefromstring("d4d5"))
@assert !extboard.repetitionRuleDraw 
domove!(extboard, Chess.movefromstring("f6e6"))
@assert !extboard.repetitionRuleDraw

# reverse both moves
domove!(extboard, Chess.movefromstring("d5d4"))
@assert !extboard.repetitionRuleDraw
domove!(extboard, Chess.movefromstring("e6f6"))
# second visit of position -> ok
@assert !extboard.repetitionRuleDraw

# white move, black move
domove!(extboard, Chess.movefromstring("d4d5"))
@assert !extboard.repetitionRuleDraw
domove!(extboard, Chess.movefromstring("f6e6"))
@assert !extboard.repetitionRuleDraw

# reverse both moves
domove!(extboard, Chess.movefromstring("d5d4"))
@assert !extboard.repetitionRuleDraw
domove!(extboard, Chess.movefromstring("e6f6"))
# third visit of position -> draw
@assert extboard.repetitionRuleDraw

println("Third Repetition")
board

Initial State


HTML{String}("Board:<div class=\"chessboard\"><style soped=\"true\">.dark-square {\n    fill: #c3936a;\n    stroke-width: 0;\n}\n.light-square {\n    fill: #ffd7a6;\n    stroke-width: 0;\n}\n.highlight {\n    fill: #47d18b;\n}\n@media (prefers-color-scheme: dark) {\n    .dark-square {\n        fill: #00796b;\n    }\n    .light-square {\n        fill: #3ca59b;\n    }\n    .highlight {\n        fill: #d23600;\n    }\n}\n</style><svg style=\"float: left; margin-right: 20px\" viewBox=\"0 0 8 8\" height=\"280\" width=\"280\"><g><g><rect x=\"0\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/br.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"7\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wr.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"0\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bn.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"1\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"2\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"3\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"4\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"5\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"6\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"7\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wn.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bb.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"5\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" 

Third Repetition


HTML{String}("Board:<div class=\"chessboard\"><style soped=\"true\">.dark-square {\n    fill: #c3936a;\n    stroke-width: 0;\n}\n.light-square {\n    fill: #ffd7a6;\n    stroke-width: 0;\n}\n.highlight {\n    fill: #47d18b;\n}\n@media (prefers-color-scheme: dark) {\n    .dark-square {\n        fill: #00796b;\n    }\n    .light-square {\n        fill: #3ca59b;\n    }\n    .highlight {\n        fill: #d23600;\n    }\n}\n</style><svg style=\"float: left; margin-right: 20px\" viewBox=\"0 0 8 8\" height=\"280\" width=\"280\"><g><g><rect x=\"0\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/br.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"7\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wr.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"0\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bn.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"1\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"2\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"3\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"4\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"5\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"6\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"7\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wn.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bb.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"5\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" 

Durchführung des gleichen Tests in dem implementieren Spiel.

In [8]:
struct MoveBot
    moves::Array{String}
end

function getNextMove(bot::MoveBot, extboard::ExtendedBoard)::Chess.Move
    return Chess.movefromstring(pop!(bot.moves))
end

getNextMove (generic function with 1 method)

In [15]:
@nbinclude "2.0 - Gameplay.ipynb"

saveGame (generic function with 1 method)

In [17]:
fen = "rnb1kbnr/pppp1ppp/5q2/8/3QP3/2P5/PP3PPP/RNB1KBNR w KQkq -"

whiteBot = MoveBot(reverse(["d4d5", "d5d4", "d4d5", "d5d4"]))
blackBot = MoveBot(reverse(["f6e6", "e6f6", "f6e6", "e6f6"]))
        
@assert play(white = whiteBot, black = blackBot, fen = fen) == Draw

rnb1kbnr/pppp1ppp/5q2/8/3QP3/2P5/PP3PPP/RNB1KBNR w KQkq -


HTML{String}("Board:<div class=\"chessboard\"><style soped=\"true\">.dark-square {\n    fill: #c3936a;\n    stroke-width: 0;\n}\n.light-square {\n    fill: #ffd7a6;\n    stroke-width: 0;\n}\n.highlight {\n    fill: #47d18b;\n}\n@media (prefers-color-scheme: dark) {\n    .dark-square {\n        fill: #00796b;\n    }\n    .light-square {\n        fill: #3ca59b;\n    }\n    .highlight {\n        fill: #d23600;\n    }\n}\n</style><svg style=\"float: left; margin-right: 20px\" viewBox=\"0 0 8 8\" height=\"280\" width=\"280\"><g><g><rect x=\"0\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/br.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"7\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wr.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"0\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bn.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"1\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"2\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"3\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"4\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"5\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"6\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"7\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wn.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bb.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"5\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" 

Last Move: Move(e6f6)
Saved game to games/2023-05-27T12-29-09.065.pgn
Draw (Repetition Rule)


Weitere Tests, z.B. ob die KIs ein Wiederholungs-Unentschieden erkennen, werden in den entsprechenden Notebooks durchgeführt.

***